# Bước 2c: Tìm kiếm Top-K bằng Faiss (Whitened L2) trên Database Train đã “làm trắng”

## Mục tiêu của script
Script này thực hiện pipeline tìm kiếm (retrieval) như sau:
- **Database nền (Train)**: đã được **làm trắng (whitened)** từ Bước 2b.
- **Database truy vấn (Test)**: vẫn là **vector gốc** (chưa whiten).
- **Ma trận whitening (mean + W)**: lấy từ Bước 2a.

Với mỗi truy vấn trong Test:
1. **Whiten vector truy vấn** (theo từng layer) bằng `mean` và `W` tương ứng.
2. Dùng **Faiss IndexFlatL2** để tính **khoảng cách L2** giữa truy vấn (đã whiten) và DB train (đã whiten) theo từng layer.
3. **Cộng dồn khoảng cách L2 qua 12 layer** để ra “tổng khoảng cách” cuối cùng.
4. Lấy **Top-K** mẫu có **tổng khoảng cách nhỏ nhất**.
5. Lưu kết quả ra file JSON gồm `query_info` và danh sách `neighbors`.

## Điều kiện chạy trong Colab
- Đã cài:
  - `!pip install faiss-gpu` (hoặc `faiss-cpu`)
- Đã **Restart Runtime** sau khi cài Faiss.

## Thư viện sử dụng
- `faiss`: xây dựng index và tìm kiếm L2 nhanh.
- `torch`: load vector `.pt`, whitening query (matmul), no_grad.
- `os`: đường dẫn file và tạo thư mục output.
- `json`: load metadata và lưu kết quả search.
- `numpy`: thao tác mảng khoảng cách và sort Top-K.
- `tqdm.auto`: progress bar cho build index và tìm kiếm.
- `sys`: import sẵn (trong code hiện tại **chưa dùng**).

## 1) Cấu hình chính
- `K = 5`  
  Số lượng hàng xóm gần nhất cần lấy (Top-K).

- `D = 2304`  
  Kích thước của mỗi vector (dimension).

- `NUM_LAYERS = 12`  
  Số layer vector (layer_1 → layer_12).

### Chạy trên CPU để tránh lỗi
Script cố tình **khóa CPU**:
- `device = torch.device("cpu")`
- `use_gpu = False`

Ý nghĩa:
- Whitening query vẫn dùng PyTorch nhưng chạy CPU.
- Faiss cũng chạy CPU (không dùng GPU).

## 2) Thiết lập đường dẫn I/O

### Thư mục
- `db_cache_dir`: nơi chứa các file database vector và metadata.
- `output_results_dir`: nơi lưu file kết quả search (`search_results/`).

### Input files
1) **Train DB (đã whiten)** — từ Bước 2b  
- `combined_train_db_vectors_v2_WHITENED.pt`  
- `combined_train_db_metadata_v2_inner_content.json`

2) **Test DB (gốc)** — từ Bước 1  
- `combined_TEST_db_vectors_v2_inner_content.pt`  
- `combined_TEST_db_metadata_v2_inner_content.json`

3) **Whitening matrices** — từ Bước 2a  
- `mahalanobis_data_v2_inner_content.pt`

### Output file
- `search_results_top{K}_v2_WHITENED_L2.json`  
Chứa danh sách kết quả cho toàn bộ query trong Test.

## 3) Tải Database Nền (Train đã whiten)
- Load tensor:
  - `train_vectors_db_whitened = torch.load(..., map_location='cpu')`
  - Shape: `[N, 12, 2304]`
- Load metadata:
  - `train_metadata_db = json.load(...)`
- Kiểm tra số lượng:
  - `N_samples = len(train_metadata_db)`
  - `len(train_vectors_db_whitened) == N_samples`

Nếu load lỗi → dừng chương trình.

## 4) Tải Database Truy vấn (Test gốc)
- Load tensor:
  - `test_vectors_db_original = torch.load(..., map_location='cpu')`
  - Shape: `[M, 12, 2304]`
- Load metadata:
  - `test_metadata_db = json.load(...)`
- Kiểm tra số lượng:
  - `M_samples = len(test_metadata_db)`
  - `len(test_vectors_db_original) == M_samples`

Nếu load lỗi → dừng chương trình.

## 5) Tải Ma trận Whitening (mean và W)
- Load:
  - `mahalanobis_data = torch.load(..., map_location=device)`
- Dữ liệu chứa:
  - `mahalanobis_data['means']['layer_k']` shape `(2304,)`
  - `mahalanobis_data['whitening_matrices']['layer_k']` shape `(2304, 2304)`

Trong code hiện tại, `device` là CPU nên toàn bộ mean/W ở CPU.

## 6) Xây dựng Faiss IndexFlatL2 cho 12 layer
Mục tiêu:
- Mỗi layer có một index riêng để search L2 nhanh.

Với mỗi `layer_idx`:
1. Trích vector Train layer đó:
   - `layer_train_vectors = train_vectors_db_whitened[:, layer_idx, :]`
   - Shape: `[N, 2304]`
2. Đưa sang numpy float32:
   - `.numpy().astype('float32')`
3. Tạo index L2:
   - `index = faiss.IndexFlatL2(D)`
4. Thêm toàn bộ vector vào index:
   - `index.add(layer_train_vectors)`
5. Lưu index vào `faiss_indexes`

Sau khi xong:
- `del train_vectors_db_whitened` để giải phóng RAM.

> Ghi chú: `IndexFlatL2` là index brute-force (chính xác), có thể chậm nếu N rất lớn, nhưng đơn giản và chuẩn.

## 7) Thực hiện tìm kiếm cho từng query

### Ý tưởng tổng quát
Với mỗi query (Test):
- Tạo mảng `total_distances_L2` kích thước `[N_samples]` để **cộng dồn khoảng cách** qua 12 layer.
- Lặp từng layer:
  1. Whiten query layer đó: `(q - mean) @ W^T`
  2. Faiss search trên index layer đó để lấy khoảng cách với toàn bộ DB.
  3. Chuyển từ khoảng cách bình phương sang L2 thật: `sqrt(d^2)`
  4. Cộng vào `total_distances_L2`

Cuối cùng:
- Sort `total_distances_L2` và lấy Top-K nhỏ nhất.

### 7.1 Khởi tạo cho mỗi query
- `query_metadata = test_metadata_db[query_idx]`
- `query_vector_original = test_vectors_db_original[query_idx]`  
  Shape: `[12, 2304]`

Tạo mảng cộng dồn:
- `total_distances_L2 = np.zeros(N_samples, dtype='float32')`

### 7.2 Lặp qua 12 layer của query
Với mỗi `layer_idx`:
- `layer_name = f"layer_{layer_idx+1}"`

#### (1) Whitening query theo layer
- Lấy:
  - `mean = mahalanobis_data['means'][layer_name]`
  - `W = mahalanobis_data['whitening_matrices'][layer_name]`
- Vector query layer gốc:
  - `query_vec_original_layer = query_vector_original[layer_idx]` shape `(2304,)`
- Center:
  - `query_centered = query_vec_original_layer - mean`
- Whiten:
  - `query_whitened = query_centered @ W.T` shape `(2304,)`

Chuyển sang numpy để Faiss xử lý:
- `query_whitened_np` shape `(1, 2304)` float32

#### (2) Search Faiss trên layer tương ứng
- Lấy index:
  - `index_for_this_layer = faiss_indexes[layer_idx]`

Search:
- `index.search(query_whitened_np, N_samples)`

Ở đây `k = N_samples` nghĩa là:
- lấy **kết quả cho toàn bộ DB** (tức brute-force trả ra danh sách full sorted theo khoảng cách).

Faiss trả:
- `D_matrix_L2_squared`: khoảng cách L2 bình phương, shape `(1, N_samples)`
- `I_matrix`: chỉ số tương ứng (sorted), shape `(1, N_samples)`

#### (3) “Giải-sắp-xếp” (unsort) để khớp đúng index gốc
Faiss trả kết quả theo thứ tự tăng dần khoảng cách, nên cần đưa về đúng vị trí `[0..N-1]` để cộng dồn:
- `distances_L2_squared_sorted = D_matrix_L2_squared[0]`
- `indices_sorted = I_matrix[0]`

Tạo mảng unsorted:
- `unsorted_distances_L2_squared[indices_sorted] = distances_L2_squared_sorted`

Sau đó lấy L2 thật:
- `distances_L2 = sqrt(unsorted_distances_L2_squared)`

#### (4) Cộng dồn khoảng cách qua layer
- `total_distances_L2 += distances_L2`

Kết quả sau 12 layer:
- `total_distances_L2[j]` = tổng L2 của query với vector train thứ `j` trên toàn bộ layer.

### 7.3 Lấy Top-K hàng xóm
- `match_indices = argsort(total_distances_L2)[:K]`
- `match_distances = total_distances_L2[match_indices]`

Tạo danh sách `top_k_matches` gồm:
- `match_db_index`: chỉ số vector trong DB train
- `total_L2_distance_whitened`: tổng khoảng cách L2 (cộng 12 layer)
- `match_metadata`: metadata tương ứng từ `train_metadata_db`

### 7.4 Ghi kết quả cho từng query
Mỗi query sẽ được lưu dưới dạng:
- `query_info`: metadata của query
- `neighbors`: danh sách K kết quả gần nhất

Tất cả query được gom vào:
- `all_search_results`

## 8) Lưu kết quả ra file JSON
- Ghi:
  - `json.dump(all_search_results, indent=4)`
- File output:
  - `search_results_top{K}_v2_WHITENED_L2.json`


## Kết quả đầu ra
File JSON kết quả có dạng:
- Danh sách length = `M_samples`
- Mỗi phần tử gồm:
  - `query_info`: thông tin query
  - `neighbors`: Top-K kết quả (index + score + metadata)

## Ghi chú hiệu năng (quan trọng)
- Việc gọi:
  - `index.search(query, N_samples)`
  là rất nặng vì lấy **toàn bộ** khoảng cách cho mỗi layer và mỗi query.
- Nếu `N_samples` lớn, bước này sẽ:
  - chậm
  - tốn RAM/CPU

Nếu muốn tối ưu, có thể:
- chỉ search `k` nhỏ hơn (ví dụ 1000), rồi cộng dồn trên tập ứng viên
- hoặc dùng IVF/HNSW trong Faiss để approximate search
- hoặc gộp vector 12-layer thành một vector lớn rồi index một lần (tuỳ thiết kế)


In [ ]:
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 110.0 MB/s eta 0:00:00


In [ ]:
# --- CELL 2: IMPORT & CODE CHÍNH ---
# Giả định bạn đã chạy "!pip install faiss-gpu" trong Cell 1
# và đã KHỞI ĐỘNG LẠI THỜI GIAN CHẠY (Runtime -> Restart Runtime)

import faiss
import torch
import os
import json
import numpy as np
from tqdm.auto import tqdm
import sys

print("--- Bắt đầu Bước 2c: Tìm kiếm (Whitened L2 - Faiss) ---")
print("Phương pháp: 'Làm trắng' query và tìm L2 (Euclidean) trên DB đã 'làm trắng'.")

# --- 1. CẤU HÌNH ---
K = 5 # Số lượng vector tương đồng gần nhất cần tìm (Top-K)
D = 2304 # Kích thước của mỗi vector
NUM_LAYERS = 12

# SỬA LỖI: Buộc script chạy trên CPU
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# use_gpu = torch.cuda.is_available()
device = torch.device("cpu")
use_gpu = False
print(f"Sử dụng thiết bị: {device} (Faiss GPU: {use_gpu})")


# --- 2. THIẾT LẬP CÁC ĐƯỜNG DẪN ---
drive_base_path = '/content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep'
db_cache_dir = os.path.join(drive_base_path, 'cached_databases')
output_results_dir = os.path.join(drive_base_path, 'search_results')
os.makedirs(output_results_dir, exist_ok=True)

# --- Các file INPUT ---

# 1. Database NỀN (ĐÃ LÀM TRẮNG) - (Từ Bước 2b)
train_vectors_file = os.path.join(db_cache_dir, 'combined_train_db_vectors_v2_WHITENED.pt')
train_metadata_file = os.path.join(db_cache_dir, 'combined_train_db_metadata_v2_inner_content.json') # Metadata gốc

# 2. Database TRUY VẤN (GỐC) - (Từ Bước 1)
test_vectors_file = os.path.join(db_cache_dir, 'combined_TEST_db_vectors_v2_inner_content.pt')
test_metadata_file = os.path.join(db_cache_dir, 'combined_TEST_db_metadata_v2_inner_content.json') # Metadata gốc

# 3. Ma trận Whitening - (Từ Bước 2a)
mahalanobis_data_file = os.path.join(db_cache_dir, 'mahalanobis_data_v2_inner_content.pt')

# --- File OUTPUT (Tên file mới) ---
results_file = os.path.join(output_results_dir, f'search_results_top{K}_v2_WHITENED_L2.json') # Tên file kết quả mới

print(f"DB 'Làm trắng' (Train): {train_vectors_file}")
print(f"DB Gốc (Test): {test_vectors_file}")
print(f"File ma trận: {mahalanobis_data_file}")
print(f"Kết quả sẽ lưu tại: {results_file}")

# --- 3. TẢI DATABASE NỀN (ĐÃ LÀM TRẮNG) ---
print("\nĐang tải Database Nền (ĐÃ LÀM TRẮNG)...")
try:
    train_vectors_db_whitened = torch.load(train_vectors_file, map_location='cpu') # [N, 12, 2304]
    with open(train_metadata_file, 'r', encoding='utf-8') as f:
        train_metadata_db = json.load(f)
    N_samples = len(train_metadata_db)
    assert len(train_vectors_db_whitened) == N_samples
    print(f" -> Tải thành công {N_samples} vector Train đã 'làm trắng'.")
except Exception as e:
    print(f"LỖI: Không thể tải database Train 'làm trắng'. {e}")
    exit()

# --- 4. TẢI DATABASE TRUY VẤN (GỐC) ---
print("\nĐang tải Database Truy vấn (GỐC)...")
try:
    test_vectors_db_original = torch.load(test_vectors_file, map_location='cpu') # [M, 12, 2304]
    with open(test_metadata_file, 'r', encoding='utf-8') as f:
        test_metadata_db = json.load(f)
    M_samples = len(test_metadata_db)
    assert len(test_vectors_db_original) == M_samples
    print(f" -> Tải thành công {M_samples} vector Test (truy vấn).")
except Exception as e:
    print(f"LỖI: Không thể tải database Test. {e}")
    exit()

# --- 5. TẢI MA TRẬN WHITENING ---
print("\nĐang tải Ma trận Whitening (mean và W)...")
try:
    mahalanobis_data = torch.load(mahalanobis_data_file, map_location=device) # Tải thẳng lên GPU
    print(f" -> Tải thành công {len(mahalanobis_data['means'])} ma trận.")
except Exception as e:
    print(f"LỖI: Không thể tải file ma trận. {e}")
    exit()

# --- 6. XÂY DỰNG FAISS INDEX (L2) TỪ DB ĐÃ LÀM TRẮNG ---
print("\nĐang xây dựng Faiss Index (L2) cho 12 layer...")

faiss_indexes = []
for layer_idx in tqdm(range(NUM_LAYERS), desc="Xây dựng Index từng Layer"):
    # 1. Trích xuất vector [N, 2304] của layer này TỪ DB ĐÃ LÀM TRẮNG
    layer_train_vectors = train_vectors_db_whitened[:, layer_idx, :].numpy().astype('float32')

    # 2. Tạo index L2 (Euclidean)
    index = faiss.IndexFlatL2(D)

    if use_gpu:
        res = faiss.StandardGpuResources()
        index = faiss.index_cpu_to_gpu(res, 0, index)

    # 3. Thêm vector vào index
    index.add(layer_train_vectors)
    faiss_indexes.append(index)

print(" -> Xây dựng 12 Index (L2) thành công!")
# Giải phóng bộ nhớ, không cần DB train trên CPU nữa
del train_vectors_db_whitened

# --- 7. THỰC HIỆN TÌM KIẾM (NHANH) ---
print(f"\nBắt đầu tìm kiếm Top-{K} cho {M_samples} truy vấn...")

all_search_results = [] # Lưu kết quả cuối cùng

with torch.no_grad():
    for query_idx in tqdm(range(M_samples), desc="Tìm kiếm truy vấn (Faiss L2)"):
        query_metadata = test_metadata_db[query_idx]
        query_vector_original = test_vectors_db_original[query_idx].to(device) # [12, 2304]

        # Mảng [N_samples] (ví dụ: [15521]) để cộng dồn tổng khoảng cách
        total_distances_L2 = np.zeros(N_samples, dtype='float32')

        # Lặp qua 12 layer của vector truy vấn này
        for layer_idx in range(NUM_LAYERS):
            layer_name = f"layer_{layer_idx+1}"

            # 1. "Làm trắng" vector query của layer này
            mean = mahalanobis_data['means'][layer_name] # Đã ở trên GPU
            W = mahalanobis_data['whitening_matrices'][layer_name] # Đã ở trên GPU

            query_vec_original_layer = query_vector_original[layer_idx] # [2304]

            query_centered = query_vec_original_layer - mean
            query_whitened = torch.matmul(query_centered, W.T) # [2304]

            # Chuyển về numpy [1, 2304] để Faiss xử lý
            query_whitened_np = query_whitened.unsqueeze(0).cpu().numpy().astype('float32')

            # 2. Lấy index L2 của layer tương ứng
            index_for_this_layer = faiss_indexes[layer_idx]

            # 3. Tìm kiếm L2! (Tìm K=N_samples, tức là lấy hết)
            D_matrix_L2_squared, I_matrix = index_for_this_layer.search(query_whitened_np, N_samples)

            distances_L2_squared_sorted = D_matrix_L2_squared[0]
            indices_sorted = I_matrix[0]

            # "Giải-sắp-xếp" (unsort) mảng khoảng cách
            unsorted_distances_L2_squared = np.empty_like(distances_L2_squared_sorted)
            unsorted_distances_L2_squared[indices_sorted] = distances_L2_squared_sorted

            # Lấy căn bậc 2 để ra khoảng cách L2 (Euclidean) thực sự
            distances_L2 = np.sqrt(unsorted_distances_L2_squared)

            # 4. Cộng dồn vào tổng khoảng cách (mảng đã unsort)
            total_distances_L2 += distances_L2

        # 5. TÌM TOP-K (K index có tổng khoảng cách NHỎ NHẤT)
        match_indices = np.argsort(total_distances_L2)[:K]
        match_distances = total_distances_L2[match_indices]

        # 6. Xử lý kết quả
        top_k_matches = []
        for i in range(K):
            match_index = match_indices[i]
            match_score = match_distances[i]
            match_metadata = train_metadata_db[match_index]

            top_k_matches.append({
                'match_db_index': int(match_index),
                'total_L2_distance_whitened': float(match_score), # Đây là TỔNG khoảng cách L2
                'match_metadata': match_metadata
            })

        all_search_results.append({
            'query_info': query_metadata,
            'neighbors': top_k_matches
        })

print(" -> Tìm kiếm hoàn tất!")

# --- 8. LƯU KẾT QUẢ ---
print(f"\nĐang lưu {len(all_search_results)} kết quả truy vấn vào file...")
try:
    with open(results_file, 'w', encoding='utf-8') as f:
        json.dump(all_search_results, f, indent=4)
    print(f" -> Đã lưu kết quả thành công: {results_file}")
except Exception as e:
    print(f"LỖI khi lưu file kết quả: {e}")

print("\n--- BƯỚC 2c (TÌM KIẾM FAISS L2) HOÀN TẤT! ---")

--- Bắt đầu Bước 2c: Tìm kiếm (Whitened L2 - Faiss) ---
Phương pháp: 'Làm trắng' query và tìm L2 (Euclidean) trên DB đã 'làm trắng'.
Sử dụng thiết bị: cpu (Faiss GPU: False)
DB 'Làm trắng' (Train): /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/combined_train_db_vectors_v2_WHITENED.pt
DB Gốc (Test): /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/combined_TEST_db_vectors_v2_inner_content.pt
File ma trận: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/cached_databases/mahalanobis_data_v2_inner_content.pt
Kết quả sẽ lưu tại: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/search_results/search_results_top5_v2_WHITENED_L2.json

Đang tải Database Nền (ĐÃ LÀM TRẮNG)...
 -> Tải thành công 15521 vector Train đã 'làm trắng'.

Đang tải Database Truy vấn (GỐC)...
 -> Tải thành công 2410 vector Test (truy vấn).

Đang tải Ma trận Whitening (mean và W)...
 -> Tải thành công 12 ma trận.

Đang xây dựng Faiss Index (L2

Xây dựng Index từng Layer:   0%|          | 0/12 [00:00<?, ?it/s]

 -> Xây dựng 12 Index (L2) thành công!

Bắt đầu tìm kiếm Top-5 cho 2410 truy vấn...


Tìm kiếm truy vấn (Faiss L2):   0%|          | 0/2410 [00:00<?, ?it/s]

 -> Tìm kiếm hoàn tất!

Đang lưu 2410 kết quả truy vấn vào file...
 -> Đã lưu kết quả thành công: /content/drive/MyDrive/Colab Notebooks/Khoa_Luan_Tot_Nghiep/search_results/search_results_top5_v2_WHITENED_L2.json

--- BƯỚC 2c (TÌM KIẾM FAISS L2) HOÀN TẤT! ---
